<img src=https://www.factset.com/hubfs/Assets/images/factset-logo.svg width="300" align="left">

# Documents API - Global Filings API V2 - Getting Started
This notebook demonstrates basic features of the Global Filings API V2 by walking through the following steps:

1. Import Python packages 

2. Authenticating using OAuth

3. For each FactSet Global Filings endpoint, create request objects and display the results.

Additional Materials:  
 
* [FactSet Developer Portal](https://developer.factset.com/api-catalog/global-filings-api)

## 1. Import the required packages

In [2]:
from fds.sdk.utils.authentication import ConfidentialClient # To install, please use pip install fds.sdk.utils

import fds.sdk.GlobalFilings # To install, please use pip install fds.sdk.GlobalFilings==2.0.0
from fds.sdk.GlobalFilings.api import filings_api_api, meta_api
from fds.sdk.GlobalFilings.models import *
from dateutil.parser import parse as dateutil_parser
from pprint import pprint
import pandas as pd
pd.set_option("display.max_rows", None)  # Display all rows
pd.set_option("display.max_columns", None)
import requests
import os
from urllib.parse import urlparse

## 2. Create a connection object

Click [here](https://developer.factset.com/authentication) for more details on Authentication.

In [ ]:
# Basic authentication: OAuth
configuration = fds.sdk.GlobalFilings.Configuration(
    fds_oauth_client=ConfidentialClient(r"<PATH_TO_CONFIG_FILE>")
)

# grab the raw access token once
access_token = configuration.fds_oauth_client.get_access_token()
headers = {
    'Authorization': f'Bearer {access_token}'
}

output_dir = r'C:/Users/USERNAME' # make sure to replace 'USERNAME' with your actual username

# 3. Global Filings API endpoint details

These endpoints provide access to search and download filings documents from various sources.
The notebook creates a requests object and displays a dataframe for each of the following endpoints:

1. **/search** - [/global-filings/v2/search](#search)
2. **/count** - [/global-filings/v2/count](#count)
3. **meta/sources** - [/global-filings/v2/meta/sources](#reference_sources)
4. **meta/form-types** - [/global-filings/v2/meta/form-types](#reference_form_types)
5. **meta/time-zones** - [/global-filings/v2/meta/time-zones](#reference_time_zones)
6. **meta/categories** - [/global-filings/v2/meta/categories](#reference_categories)


For additional details regarding each endpoint's request parameters or response models, visit the [FactSet Global Filings API](https://developer.factset.com/api-catalog/global-filings-api) specification page.

<a id='search'></a>
## Search
Returns the filings documents within FactSet coverage along with other response fields.

In [7]:
# Enter a context with an instance of the API client
with fds.sdk.GlobalFilings.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = filings_api_api.FilingsAPIApi(api_client)
    
    api_response = api_instance.get_filings(
            ids = ["MODN-US"], 
            sources = ["EDG"], 
            start_date = "20240601",
            end_date = "20241101",
            pagination_limit = 20,
            pagination_offset = 0, 
            time_zone = "America/New_York", 
            sort = ["filingsDateTime"],
            categories = ["CN:US"],
            primary_id = False, 
            search_text = "Officer",
            form_types = ["15-12G"]            
    )
    print(api_response)

{'data': [{'documents': [{'accession': '0001193125-24-176289',
                          'all_ids': ['MODN-US', 'MODN'],
                          'categories': ['DT:FILNS',
                                         'LN:EN',
                                         'SB:FILNS',
                                         'RN:AM',
                                         'RN:NA',
                                         'IN:COMPTC',
                                         'CN:US'],
                          'document_id': '0001193125-24-176289-1',
                          'filing_size': '10.77 KB',
                          'filings_date_time': '2024-07-08T16:06:38Z',
                          'filings_link': 'https://api.factset.com/global-filings/v2/filings?report=story&timezone=America/New_York&key=U2FsdGVkX1+bk5hdKLP7NVX6J66rc/e039Evcg2foxd/IJ8i/DYQYewQWigA0EBaKDNfhYHXGU18ZbxqsSgudiZvRLaeCMy6etcLX/LnM9g=',
                          'form_types': ['15-12G'],
                          'h

### File download

In [8]:
print("\nFile download:")
for entry in api_response['data']:
    for item in entry.get('documents', []):
        url = item['filings_link']
        try:
            # pass the OAuth bearer token in headers
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Request failed: {e}")
            continue

        parsed_url = urlparse(url)
        filename = f"{item['document_id']}_{os.path.basename(parsed_url.path)}.html"
        path = os.path.join(output_dir, filename)

        try:
            with open(path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded: {filename}")
        except IOError as e:
            print(f"Failed to write to file: {e}")


File download:
Downloaded: 0001193125-24-176289-1_filings.html


<a id='count'></a>
## Count
Returns the count of filings documents along with other response fields.

In [13]:
# Enter a context with an instance of the API client
with fds.sdk.GlobalFilings.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = filings_api_api.FilingsAPIApi(api_client)

    ids = ["MODN-US"] 
    sources = ["EDG"]
    start_date = "20240601" 
    end_date = "20241101" 

    api_response = api_instance.get_count(ids, sources, start_date=start_date, end_date=end_date)
    results = pd.DataFrame(api_response.to_dict()['data'])
    display(results)


,request_id,source,count
0,MODN-US,EDG,35


<a id='reference_sources'></a>
## Sources
Retrieves and delivers a comprehensive list of all available sources.

In [17]:
# Enter a context with an instance of the API client
with fds.sdk.GlobalFilings.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = meta_api.MetaApi(api_client)

    api_response = api_instance.get_sources()
    results = pd.DataFrame(api_response.to_dict()['data'])
    display(results)

,source,description
0,ASXD,ASX Company Announcements - Delayed
1,EDG,EDGAR+
2,EDJ,EDINET
3,FBLK,FactSet Blackline Report
4,FFR,FactSet Annuals & Interims
5,HNX,HNX News
6,SDR,SEDAR
7,SENS,JSE SENS
8,TWSE,TWSE Company Announcements
9,SDRP,SEDAR PLUS


<a id='reference_form_types'></a>
## Form-types
Retrieves and delivers a comprehensive list of all available formTypes.

In [18]:
# Enter a context with an instance of the API client
with fds.sdk.GlobalFilings.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = meta_api.MetaApi(api_client)

    api_response = api_instance.get_formtype()
    results = pd.DataFrame(api_response.to_dict()['data'])
    display(results)

,source,form_type,description
0,EDGAR,1,(1) Registration as a national securities exch...
1,EDGAR,1-A,(1-A) Offering statement
2,EDGAR,1-A-W,(1-A-W) No description available
3,EDGAR,1-A-W/A,(1-A-W/A) Amendment to Form 1-A-W
4,EDGAR,1-A/A,(1-A/A) Amendment to the 1-A
5,EDGAR,1-E,(1-E) Notification under Regulation E
6,EDGAR,1-E/A,(1-E/A) Amendment to the 1-E
7,EDGAR,1-F,(1-F) Notification under Regulation F
8,EDGAR,1/A,(1/A) Amendment to Form 1
9,EDGAR,10,(10) Registration of securities


<a id='reference_time_zones'></a>
## Time-zones
Retrieves and delivers a comprehensive list of all available timeZones.

In [19]:
# Enter a context with an instance of the API client
with fds.sdk.GlobalFilings.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = meta_api.MetaApi(api_client)

    api_response = api_instance.get_timezone()
    results = pd.DataFrame(api_response.to_dict()['data'])
    display(results)

,time_zone
0,ACT
1,AUS Eastern Standard Time
2,Africa/Abidjan
3,Africa/Accra
4,Africa/Addis_Ababa
5,Africa/Algiers
6,Africa/Asmara
7,Africa/Bamako
8,Africa/Bangui
9,Africa/Banjul


<a id='reference_categories'></a>
## Categories
Retrieves and delivers a comprehensive list of all available categories.

In [20]:
# Enter a context with an instance of the API client
with fds.sdk.GlobalFilings.ApiClient(configuration) as api_client:
    # Create an instance of the API class
    api_instance = meta_api.MetaApi(api_client)

    api_response = api_instance.getcategories()
    results = pd.DataFrame(api_response.to_dict()['data'])
    display(results)

,subject,category,description
0,SB,SB:ANLCH,Analysts Revisions
1,SB,SB:BNKRPT,Bankruptcy
2,SB,SB:COMM,Commodities
3,SB,SB:COARLS,Company Announcements and Releases
4,SB,SB:CORPS,Corporate Actions
5,SB,SB:DEFIN,Debt and Equity Financing
6,SB,SB:DIVS,Dividends
7,SB,SB:ERNS,Earnings and Recommendations
8,SB,SB:ECON,Economy and Economic Indicators
9,SB,SB:ESG,"Environmental, Social and Governance"


## Next Steps


* **Documentation for the APIs discussed above can be found within the [Developer Portal](https://developer.factset.com/api-catalog/global-filings-api)**


* **Please direct any requests or questions to your CTS Specialist or file an [Issue Tracker](issuetracker.factset.com) under the "Juypter Notebook" sub-category.**

> Disclaimer: *The data in this Notebook is proprietary to FactSet Research Systems Inc. and its affiliates ("FactSet") and its third-party data suppliers ("Suppliers").  The data is provided "as is," for internal use in accordance with the terms of the license agreement or trial agreement in place between you and FactSet, and FactSet and its Suppliers disclaim all express and implied warranties, including, without limitation, any express or implied warranty of accuracy, satisfactory quality, or fitness for a particular purpose.*